In [10]:
import anndata
import scanpy as sc
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pandas as pd
import umap
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from keras.callbacks import EarlyStopping

adata = sc.read('/tmp/work/RCproject_code/sce_export.h5ad')

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [11]:
# generate numerical values for each batch category
from sklearn.preprocessing import LabelEncoder
# set up categories variable
categories = adata.obs['batch']
# Create a LabelEncoder instance
label_encoder = LabelEncoder()
# Fit and transform the categories to integers
numerical_categories = label_encoder.fit_transform(categories)
numerical_categories.shape

(450,)

In [12]:
#normalizaiton
gene_expression_data = adata.layers['logcounts']
from sklearn.preprocessing import MinMaxScaler
# Min-max normalization
scaler = MinMaxScaler()
gene_expression_data = scaler.fit_transform(gene_expression_data)
number_samples = adata.shape[0]
number_genes = adata.shape[1]
input_dim = number_genes


In [13]:
input_shape = gene_expression_data[0,:].shape
encoding_dim = 64  # This is the size of the encoded representation

In [14]:
# Define the encoder function
def build_encoder():    
    model = models.Sequential()    

    # Input layer
    model.add(layers.Input(shape=input_shape)) 

    # 1st hidden layer
    model.add(layers.Dense(256, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())    
    model.add(layers.Dropout(0.3))

    # 2nd hidden layer
    model.add(layers.Dense(128, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())    
    model.add(layers.Dropout(0.3))

    # 3rd hidden layer (encoded representation)
    model.add(layers.Dense(encoding_dim, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    return model 

In [15]:
# Define the decoder function
def build_decoder():    
    model = models.Sequential()

    # Input layer with the size of the encoded representation
    model.add(layers.Input(shape=(encoding_dim,)))  

    # 1st hidden layer
    model.add(layers.Dense(64, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # 2nd hidden layer
    model.add(layers.Dense(128, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # 3rd hidden layer
    model.add(layers.Dense(256, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # Output layer (reconstruction of the original input)
    model.add(layers.Dense(input_shape[0], activation='sigmoid'))  # Use 'sigmoid' if input is normalized to [0, 1]

    return model 

In [16]:
# def build_decoder():    
#     model = models.Sequential()

#     # Input layer with the size of the encoded representation
#     model.add(layers.Input(shape=(encoding_dim,)))  

#     # 1st hidden layer
#     model.add(layers.Dense(64, activation='linear'))  
#     model.add(layers.LeakyReLU())    
#     model.add(layers.BatchNormalization())
#     #model.add(layers.Dropout(0.3))


#     # 2nd hidden layer
#     model.add(layers.Dense(128, activation='linear'))  
#     model.add(layers.LeakyReLU())    
#     model.add(layers.BatchNormalization())
#     #model.add(layers.Dropout(0.3))

#     # 3rd hidden layer
#     model.add(layers.Dense(256, activation='linear'))  
#     model.add(layers.LeakyReLU())    
#     model.add(layers.BatchNormalization())
#     #model.add(layers.Dropout(0.3))

#     # Output layer (reconstruction of the original input)
#     model.add(layers.Dense(input_shape[0], activation='sigmoid'))  # Use 'sigmoid' if input is normalized to [0, 1]

#     return model 

# # Print the model summary to check the architecture

# decoder = build_decoder()

# #decoder.summary()

In [17]:
def build_domain_classifier(input_shape, num_domains):
    model = models.Sequential()
    
    # Input layer (matching the output shape of the feature extractor)
    model.add(layers.Input(shape=input_shape))
    
    # Hidden layer 1
    model.add(layers.Dense(128, activation=None))
    model.add(layers.LeakyReLU()) 
    model.add(layers.BatchNormalization())
    
    # Hidden layer 2
    model.add(layers.Dense(64, activation=None))
    model.add(layers.LeakyReLU()) 
    model.add(layers.BatchNormalization())
    
    # Hidden layer 3
    model.add(layers.Dense(32, activation=None))
    model.add(layers.LeakyReLU())
    model.add(layers.BatchNormalization())
    
    # Output layer (softmax for multi-class classification)
    model.add(layers.Dense(num_domains, activation='softmax'))  # num_domains is the number of classes

    return model

# Example usage
input_shape = (64,)  # Example input shape (output from encoder)
num_domains = 9  # Number of different domains
domain_classifier = build_domain_classifier(input_shape, num_domains)

# Print the model summary to check the architecture

#domain_classifier.summary()

In [18]:
# Build the encoder and decoder
encoder = build_encoder()
decoder = build_decoder()

# Create the full autoencoder model
input_layer = layers.Input(shape=input_shape)  # This shape should match the original input data
encoded_output = encoder(input_layer)  # This should output shape (None, 64)
decoded_output = decoder(encoded_output)  # This should receive the encoded shape (None, 64)

# Create the full autoencoder model
autoencoder = models.Model(inputs=input_layer, outputs=decoded_output)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = autoencoder.fit(gene_expression_data, gene_expression_data,
                          epochs=100,
                          batch_size=32,
                          shuffle=True,
                          validation_split=0.2,
                          callbacks=[early_stopping])

# Now you can use the autoencoder to reconstruct data and extract the encoded features

Epoch 1/100


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "model_1"

In [ ]:
# Get the low-dimensional representation
low_dimensional_representation = encoder.predict(gene_expression_data)

In [ ]:
##

In [ ]:
# Build autoencoder

input_layer = layers.Input(shape=input_shape)
encoded_output = encoder(input_layer)
decoded_output = decoder(encoded_output)

# Create the full autoencoder model
autoencoder = models.Model(inputs=input_layer, outputs=decoded_output)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')  # Use appropriate loss based on your data

# Train the autoencoder
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = autoencoder.fit(gene_expression_data, gene_expression_data,
                          epochs=100,
                          batch_size=32,
                          shuffle=True,
                          validation_split=0.2,
                          callbacks=[early_stopping])

In [ ]:
# Get the low-dimensional representation
low_dimensional_representation = encoder.predict(gene_expression_data)

In [ ]:
categories =  adata.obs['batch']
# categories =  adata.obs['Platform']

# Step 3: Run UMAP
umap_model = umap.UMAP(n_neighbors=15,   # Controls local vs global structure
                        min_dist=0.1,    # Controls how tightly UMAP packs points together
                        metric='euclidean')  # Distance metric to use
umap_result = umap_model.fit_transform(low_dimensional_representation)

# Step 4: Create a scatter plot with labels
plt.figure(figsize=(10, 8))

# Define a colormap
unique_categories = np.unique(categories)
cmap = ListedColormap(plt.cm.viridis(np.linspace(0, 1, len(unique_categories))))

# Create a scatter plot with color coding based on categories
scatter = plt.scatter(umap_result[:, 0], umap_result[:, 1], 
                      c=pd.Categorical(categories).codes, alpha=0.5, cmap=cmap)

# Add a title and labels
plt.title('UMAP Visualization of Data Matrix with Categories')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')

# Create a legend
# Create a legend with unique categories
handles = []
for i, cat in enumerate(unique_categories):
    handles.append(plt.Line2D([0], [0], marker='o', color='w', label=cat,
                                markerfacecolor=cmap(i), markersize=10))

# Add legend to the plot
plt.legend(handles=handles, title="Categories")

plt.grid()
plt.show()

In [19]:
from keras import layers, models
from keras.callbacks import EarlyStopping

input_shape = gene_expression_data[0, :].shape  # This should be (12165,) if that's the feature count
encoding_dim = 64  # This is the size of the encoded representation

# Define the encoder function
def build_encoder():    
    model = models.Sequential()    

    # Input layer
    model.add(layers.Input(shape=input_shape)) 

    # 1st hidden layer
    model.add(layers.Dense(256, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())    
    model.add(layers.Dropout(0.3))

    # 2nd hidden layer
    model.add(layers.Dense(128, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())    
    model.add(layers.Dropout(0.3))

    # 3rd hidden layer (encoded representation)
    model.add(layers.Dense(encoding_dim, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    return model 

# Define the decoder function
def build_decoder():    
    model = models.Sequential()

    # Input layer with the size of the encoded representation
    model.add(layers.Input(shape=(encoding_dim,)))  

    # 1st hidden layer
    model.add(layers.Dense(64, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # 2nd hidden layer
    model.add(layers.Dense(128, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # 3rd hidden layer
    model.add(layers.Dense(256, activation='linear'))  
    model.add(layers.LeakyReLU())    
    model.add(layers.BatchNormalization())

    # Output layer (reconstruction of the original input)
    model.add(layers.Dense(input_shape[0], activation='sigmoid'))  # Use 'sigmoid' if input is normalized to [0, 1]

    return model 

# Build the encoder and decoder
encoder = build_encoder()
decoder = build_decoder()

# Create the full autoencoder model
input_layer = layers.Input(shape=input_shape)  # This shape should match the original input data
encoded_output = encoder(input_layer)  # This should output shape (None, 64)
decoded_output = decoder(encoded_output)  # This should receive the encoded shape (None, 64)

# Create the full autoencoder model
autoencoder = models.Model(inputs=input_layer, outputs=decoded_output)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = autoencoder.fit(gene_expression_data, gene_expression_data,
                          epochs=100,
                          batch_size=32,
                          shuffle=True,
                          validation_split=0.2,
                          callbacks=[early_stopping])

# Now you can use the autoencoder to reconstruct data and extract the encoded features

Epoch 1/100
12/12 [==============================] - 2s 74ms/step - loss: 0.0552 - val_loss: 0.0148
Epoch 2/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0120 - val_loss: 0.0148
Epoch 3/100
12/12 [==============================] - 1s 60ms/step - loss: 0.0108 - val_loss: 0.0148
Epoch 4/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0097 - val_loss: 0.0148
Epoch 5/100
12/12 [==============================] - 1s 62ms/step - loss: 0.0091 - val_loss: 0.0148
Epoch 6/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0087 - val_loss: 0.0148
Epoch 7/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0084 - val_loss: 0.0146
Epoch 8/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0082 - val_loss: 0.0143
Epoch 9/100
12/12 [==============================] - 1s 61ms/step - loss: 0.0085 - val_loss: 0.0138
Epoch 10/100
12/12 [==============================] - 1s 60ms/step - loss: 0.0081 - val_loss: 0.0131